# Ejercicio 11 : Asistente RAG Conversacional

## Objetivo de la práctica

Construir un asistente que:

1. Recibe una pregunta del usuario
2. Recupera texto relevante de un corpus (ej. libro de Baeza-Yates)
3. Genera una respuesta basada en los documentos encontrados
4. Mantiene el historial de conversación


## Parte 0: Librerías necesarias
- openai
- faiss-cpu
- sentence-transformers

In [1]:
import pymupdf
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI

/Users/paullora/Desktop/EPN/ir/ir-2025A/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parte 1: Carga del corpus

Aquí se debe cargar el corpus con los documentos en PDF.

- Libro de Stanford
- Libro BM25
- Paper: Marcia Bates (1989). The design of browsing and berrypicking techniques for the online search interface

In [2]:
doc_standford = pymupdf.open("../data/irbookonlinereading.pdf")
doc_bm25 = pymupdf.open("../data/foundations_bm25_review.pdf")
doc_bates = pymupdf.open("../data/bates1989.pdf")

## Parte 2: Procesamiento del Corpus

Aquí se debe obtener el corpus procesado. El corpus estará formado por documentos que corresponden a las secciones (o subsecciones) de los libros. Cada documento debe indicar a qué libro corresponde, así como las páginas en las que está dentro de ese libro.

Recuerden que los documentos procesados no deben contener textos o caracteres ajenos al tema del que tratan.  

In [3]:
# Libro de Standford
# Obtiene paginas de las secciones del libro
standford_book_dict = doc_standford[4]
lines = standford_book_dict.get_text("text").split('\n')
lines = lines[7:]
pages = [lines[i] for i in range(0, len(lines), 3)]

page_limits = list(map(int, pages))

corpus_standford_book = []

for page in doc_standford:
    text = page.get_text()
    if text:
        corpus_standford_book.append(text)

corpus_standford_book = corpus_standford_book[37:517]

groups = []

for i in range(len(page_limits) - 1):
    start = page_limits[i] - 1  # -1 because Python uses 0-based indexing
    end = page_limits[i+1] - 1  # up to but not including this index
    groups.append(corpus_standford_book[start:end])

# Optionally, add the last group (from last page limit to end)
groups.append(corpus_standford_book[page_limits[-1] - 1:])

# Join each group into a single string
group_texts = [' '.join(group) for group in groups]

corpus_standford_book_df = pd.DataFrame(group_texts, columns=['raw'])

corpus_standford_book_df["clean_text"] = corpus_standford_book_df["raw"].str.replace("Online edition (c)\n2009 Cambridge UP", "")
corpus_standford_book_df["clean_text"] = corpus_standford_book_df["clean_text"].str.replace("DRAFT! © April 1, 2009 Cambridge University Press. Feedback welcome.", "")
corpus_standford_book_df["clean_text"] = corpus_standford_book_df["clean_text"].str.replace("\n", " ")
corpus_standford_book_df["clean_text"] = corpus_standford_book_df["clean_text"].str.strip()
corpus_standford_book_df["clean_text"] = corpus_standford_book_df["clean_text"].str.replace(r"^\d+\s*", "", regex=True)

corpus_standford_book_df

,raw,clean_text
0,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,1 Boolean retrieval The meaning of the term in...
1,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,2 The term vocabulary and postings lists Recal...
2,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,3 Dictionaries and tolerant retrieval In Chapt...
3,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"4 Index construction In this chapter, we look ..."
4,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,5 Index compression Chapter 1 introduced the d...
5,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"6 Scoring, term weighting and the vector space..."
6,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,7 Computing scores in a complete search system...
7,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,8 Evaluation in information retrieval We have ...
8,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,9 Relevance feedback and query expansion In mo...
9,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,10 XML retrieval Information retrieval systems...


In [4]:
# Libro de BM25
corpus_bm25_book = []

page_limits = [4, 6, 17, 41, 47, 54]

for page in doc_bm25:
    text = page.get_text()
    if text:
        corpus_bm25_book.append(text)

corpus_bm25_book = corpus_bm25_book[:-5]

groups = []

for i in range(len(page_limits) - 1):
    start = page_limits[i] - 1  # -1 because Python uses 0-based indexing
    end = page_limits[i+1] - 1  # up to but not including this index
    groups.append(corpus_bm25_book[start:end])

# Optionally, add the last group (from last page limit to end)
groups.append(corpus_bm25_book[page_limits[-1] - 1:])

# Join each group into a single string
group_texts = [' '.join(group) for group in groups]

corpus_bm25_book_df = pd.DataFrame(group_texts, columns=['raw'])
corpus_bm25_book_df["clean_text"] = corpus_bm25_book_df["raw"].str.replace("\n", " ")
corpus_bm25_book_df["clean_text"] = corpus_bm25_book_df["clean_text"].str.strip()
corpus_bm25_book_df


,raw,clean_text
0,1\nIntroduction\nThis monograph addresses the ...,1 Introduction This monograph addresses the cl...
1,2\nDevelopment of the Basic Model\n2.1\nInform...,2 Development of the Basic Model 2.1 Informati...
2,3\nDerived Models\nThe models discussed in thi...,3 Derived Models The models discussed in this ...
3,4\nComparison with Other Models\nThe ﬁrst prob...,4 Comparison with Other Models The ﬁrst probab...
4,5\nParameter Optimisation\nLike most IR models...,"5 Parameter Optimisation Like most IR models, ..."
5,6\nConclusions\nThe classical probabilistic re...,6 Conclusions The classical probabilistic rele...


In [5]:
corpus_bates_book = []
for page in doc_bates:
    text = page.get_text()
    if text:
        corpus_bates_book.append(text)

corpus_bates_book_df = pd.DataFrame(group_texts, columns=['raw'])
corpus_bates_book_df["clean_text"] = corpus_bates_book_df["raw"].str.replace("ONLINE REVIEW", "")
corpus_bates_book_df["clean_text"] = corpus_bates_book_df["raw"].str.replace("\n", " ")
corpus_bates_book_df["clean_text"] = corpus_bates_book_df["clean_text"].str.strip()
corpus_bates_book_df


,raw,clean_text
0,1\nIntroduction\nThis monograph addresses the ...,1 Introduction This monograph addresses the cl...
1,2\nDevelopment of the Basic Model\n2.1\nInform...,2 Development of the Basic Model 2.1 Informati...
2,3\nDerived Models\nThe models discussed in thi...,3 Derived Models The models discussed in this ...
3,4\nComparison with Other Models\nThe ﬁrst prob...,4 Comparison with Other Models The ﬁrst probab...
4,5\nParameter Optimisation\nLike most IR models...,"5 Parameter Optimisation Like most IR models, ..."
5,6\nConclusions\nThe classical probabilistic re...,6 Conclusions The classical probabilistic rele...


In [6]:
corpus_df = pd.concat(
    [corpus_standford_book_df, corpus_bm25_book_df, corpus_bates_book_df],
    ignore_index=True
)[["raw", "clean_text"]]

corpus_df

,raw,clean_text
0,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,1 Boolean retrieval The meaning of the term in...
1,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,2 The term vocabulary and postings lists Recal...
2,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,3 Dictionaries and tolerant retrieval In Chapt...
3,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"4 Index construction In this chapter, we look ..."
4,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,5 Index compression Chapter 1 introduced the d...
5,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"6 Scoring, term weighting and the vector space..."
6,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,7 Computing scores in a complete search system...
7,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,8 Evaluation in information retrieval We have ...
8,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,9 Relevance feedback and query expansion In mo...
9,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,10 XML retrieval Information retrieval systems...


In [7]:
stop_words = set(stopwords.words('english'))
def preprocess_doc(doc):
    words = word_tokenize(doc.lower())
    words_filtered = [word for word in words if word not in stop_words and word.isalpha()]
    return ' '.join(words_filtered)

In [8]:
corpus_df['processed'] = corpus_df['clean_text'].apply(preprocess_doc)
corpus_df

,raw,clean_text,processed
0,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,1 Boolean retrieval The meaning of the term in...,boolean retrieval meaning term information ret...
1,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,2 The term vocabulary and postings lists Recal...,term vocabulary postings lists recall major st...
2,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,3 Dictionaries and tolerant retrieval In Chapt...,dictionaries tolerant retrieval chapters devel...
3,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"4 Index construction In this chapter, we look ...",index construction chapter look construct inve...
4,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,5 Index compression Chapter 1 introduced the d...,index compression chapter introduced dictionar...
5,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"6 Scoring, term weighting and the vector space...",scoring term weighting vector space model thus...
6,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,7 Computing scores in a complete search system...,computing scores complete search system chapte...
7,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,8 Evaluation in information retrieval We have ...,evaluation information retrieval seen precedin...
8,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,9 Relevance feedback and query expansion In mo...,relevance feedback query expansion collections...
9,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,10 XML retrieval Information retrieval systems...,xml retrieval information retrieval systems of...


## Parte 3: Cálculo de Embeddings e Indexación en base de datos vectorial

Aquí, una vez que se ha calculado el embedding de cada documento, se deberá indexar este embedding en una base de datos vectorial como FAISS, ChromaDB o Pinecone

In [9]:
corpus_processed = corpus_df["processed"].tolist()
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(corpus_processed, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

index.is_trained

corpus_df['embeddings'] = embeddings.tolist()
corpus_df

,raw,clean_text,processed,embeddings
0,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,1 Boolean retrieval The meaning of the term in...,boolean retrieval meaning term information ret...,"[0.010180620476603508, -0.024825137108564377, ..."
1,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,2 The term vocabulary and postings lists Recal...,term vocabulary postings lists recall major st...,"[-0.01349974237382412, -0.08646218478679657, -..."
2,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,3 Dictionaries and tolerant retrieval In Chapt...,dictionaries tolerant retrieval chapters devel...,"[0.0313904695212841, -0.03251445293426514, -0...."
3,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"4 Index construction In this chapter, we look ...",index construction chapter look construct inve...,"[0.013512649573385715, -0.03246237337589264, -..."
4,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,5 Index compression Chapter 1 introduced the d...,index compression chapter introduced dictionar...,"[-0.014014441519975662, 0.02299860119819641, -..."
5,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,"6 Scoring, term weighting and the vector space...",scoring term weighting vector space model thus...,"[0.028798291459679604, -0.039530910551548004, ..."
6,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,7 Computing scores in a complete search system...,computing scores complete search system chapte...,"[0.03785853832960129, -0.024657947942614555, -..."
7,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,8 Evaluation in information retrieval We have ...,evaluation information retrieval seen precedin...,"[0.0004510932485572994, -0.0007152339676395059..."
8,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,9 Relevance feedback and query expansion In mo...,relevance feedback query expansion collections...,"[0.026740197092294693, -0.051840558648109436, ..."
9,Online edition (c)\n2009 Cambridge UP\nDRAFT! ...,10 XML retrieval Information retrieval systems...,xml retrieval information retrieval systems of...,"[0.01647908426821232, 0.02946741320192814, -0...."


## Parte 4: Búsqueda y obtención del contexto

En esta parte debemos definir una _query_ y buscar los documentos que más se relacionan con ella.

Estos documentos formarán el contexto que vamos a entregar al LLM.

In [10]:
query = "distancia euclidiana"
query_embedding = model.encode([query], convert_to_numpy=True)
query_embedding

array([[-2.87567191e-02,  2.41185762e-02, -1.26637202e-02,
         2.68820338e-02, -3.52203362e-02, -3.04296166e-02,
         8.07284936e-02, -3.67522240e-02,  5.70592098e-02,
        -4.87763174e-02,  8.84693787e-02, -1.43028304e-01,
        -1.66783202e-02,  3.72372009e-02, -4.10054438e-02,
        -2.66659278e-02, -1.10918740e-02,  5.79014197e-02,
        -8.12566057e-02,  1.63673796e-02,  3.73923369e-02,
         3.11759841e-02,  2.71429378e-03,  1.45310730e-01,
        -1.84971455e-03,  9.13950130e-02,  7.86481649e-02,
        -4.18036617e-03, -1.83898397e-02, -8.43591914e-02,
        -8.66919570e-03,  4.24611056e-03,  1.13893584e-04,
        -1.06601072e-02, -1.32846022e-02, -1.62804015e-02,
        -5.02117164e-02,  7.26689324e-02,  1.46853272e-02,
         3.61582376e-02, -2.73848940e-02, -1.94133576e-02,
         3.82205546e-02,  5.39974794e-02, -5.49451709e-02,
         2.42900606e-02, -4.03086245e-02,  6.58643916e-02,
         3.14030834e-02, -3.72660048e-02,  3.10178753e-0

In [11]:
top_k = 5
distances, indices = index.search(query_embedding, top_k)

## Parte 5: Generación de Respuesta

Aquí, entregamos el contexto al LLM, y él nos responde a la _query_ con una explicación en lenguaje natural.

In [ ]:
prompt = f"""
Eres una aplicacion de Retrieval Augmented Generation que siempre responde en español.
Usa el siguiente contexto para responder a la pregunta del usuario.
Si la respuesta no se encuentra en el contexto, responde "No tengo suficiente información para responder a esa pregunta".

Contexto:
{corpus_df.iloc[indices.flatten()]['clean_text'].tolist()}

Pregunta:
El usuario esta preguntando sobre "{query}".
"""
client = OpenAI(api_key="")

response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt
)
print(response.output_text)

La distancia euclidiana es una medida matemática que se utiliza para calcular la distancia “recta” o la distancia más corta entre dos puntos en un espacio vectorial. En el contexto de la representación vectorial de documentos en modelos de espacio vectorial en recuperación y clasificación de texto, la distancia euclidiana se utiliza para medir la semejanza o disimilitud entre vectores que representan documentos o consultas.

En particular, cuando los documentos y consultas están representados por vectores normalizados en longitud (es decir, vectores en la superficie de una hiperesfera), existe una relación directa entre la distancia euclidiana y la similitud coseno, otra medida común de similitud. Esto significa que para vectores normalizados, medir la distancia euclidiana es equivalente a medir la similitud coseno, siendo ambas medidas útiles para evaluar qué tan cerca o lejos están dos documentos o vectores.

Asimismo, la distancia euclidiana también se utiliza en métodos de clasific

## TTS

In [13]:
%pip install pyttsx3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pyttsx3
engine = pyttsx3.init()
engine.say(response.output_text)
engine.runAndWait()